In [22]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(42)
from sklearn.model_selection import cross_val_score
import pandas as pd
import gensim
from sklearn.cross_validation import train_test_split
import nltk
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [30]:
from keras.datasets import imdb

In [31]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17424384/17464789 [============================>.] - ETA: 0s

In [4]:
df = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')
df.shape

(1039, 186)

In [5]:
index = pd.isnull(df.formatted_text)
df.loc[index, 'formatted_text'] = ''

In [29]:
w2v = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-600.model')
w2v.init_sims(replace=True)

2017-08-28 00:50:04,345 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-08-28 00:50:13,054 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.* with mmap=None
2017-08-28 00:50:13,055 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.syn0.npy with mmap=None
2017-08-28 00:50:21,082 : INFO : setting ignored attribute syn0norm to None
2017-08-28 00:50:21,083 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.syn1neg.npy with mmap=None
2017-08-28 00:51:41,097 : INFO : setting ignored attribute cum_table to None
2017-08-28 00:51:41,395 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-08-28 00:53:35,985 : INFO : precomputing L2-norms of word weight vectors


In [24]:
num_features= 600

In [12]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [13]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    #print words.shape
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        #print mean

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [14]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [20]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [25]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-08-28 00:48:32,126 : WARNING : cannot compute similarity with no input []
2017-08-28 00:48:32,838 : WARNING : cannot compute similarity with no input []
2017-08-28 00:48:32,847 : WARNING : cannot compute similarity with no input []
2017-08-28 00:48:33,896 : WARNING : cannot compute similarity with no input []
2017-08-28 00:48:34,253 : WARNING : cannot compute similarity with no input []
2017-08-28 00:48:34,797 : WARNING : cannot compute similarity with no input []
2017-08-28 00:48:35,807 : WARNING : cannot compute similarity with no input []
2017-08-28 00:48:35,928 : WARNING : cannot compute similarity with no input []
2017-08-28 00:48:36,236 : WARNING : cannot compute similarity with no input []
2017-08-28 00:48:36,454 : WARNING : cannot compute similarity with no input []
2017-08-28 00:48:36,530 : WARNING : cannot compute similarity with no input []


CPU times: user 4.79 s, sys: 85.5 ms, total: 4.88 s
Wall time: 5.67 s


In [28]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(600, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train_word_average, train_w2v_data['extraversion_m'], validation_data=(X_test_word_average, test_w2v_data['extraversion_m']), epochs=3, batch_size=64)

ValueError: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.